In [0]:
catalog = 'operations'
schema = 'finance_staging'
table_name = 'dim_balance_sheet'

In [0]:
with open("sql/dim_balance_sheet.sql", "r") as file:
    qry = file.read()
(
    spark.sql(qry)
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{catalog}.{schema}.{table_name}")
)

## (BUILD): Balance Sheet Shcema
- Build table that provides a list of all the report labels we want to have in our balance sheet table
- For all companies in our dim company table, assign them a report label
- For reference, add the report labels report sub class grouping them to a higher standard. 
- (FUTURE): Might have to join on report label rather than report sub class

In [0]:
# spark.sql(f"""
#           select 

#             balance_sheet.report_label
#             ,company.company_bigint_key
#             ,balance_sheet.report_label_bigint_key
#             ,dates.reported_period as date_key

#           from 
#             operations.finance_staging.dim_balance_sheet balance_sheet
#           cross join 
#             operations.finance.dim_company company 
#           cross join 
#           (
#             select distinct 
#             reported_period
#             from operations.finance_staging.fact_staging_financial_statement_tbl
#             /*where reported_period in (20241231,20250331)*/
#           ) dates
#           where 
#             company.company_stock_symbol in ('AAPL','WMT','AMZN','ABBV','AMD','T','NVDA')""").createOrReplaceTempView("balance_sheet_schema")

## (BUILD): Table that provides distinct list of companies, their presented labels and what they translate to for report sub class
- (FUTURE): Switch the presented label table to have column for the respective report label (lower granularity)

In [0]:
# spark.sql(f"""
#           select distinct
#             dpl.report_label_bigint_key
#             ,dpl.presented_label_bigint_key
#             ,fsm.company_bigint_key
#             /*,fsm.presented_label_bigint_key*/
#             ,fsm.company_stock_symbol
#             ,fsm.presented_label
#           from 
#             operations.finance_staging.dim_presented_labels dpl
#           left join 
#             (select 
#               fsm.presented_label
#               ,fsm.presented_label_bigint_key
#               ,dc.company_stock_symbol
#               ,dc.company_bigint_key
#               from 
#               operations.finance_staging.fact_staging_financial_statement_tbl fsm
#               left join 
#               operations.finance.dim_company dc on dc.company_bigint_key=fsm.company_bigint_key
#               where 
#               fsm.financial_statement = 'BS'  and fsm.value_segment is null
#               and 
#               fsm.reported_period = fsm.end_reported_period 
#               and 
#               dc.company_stock_symbol in ('AAPL','WMT','AMZN','ABBV','AMD','T','NVDA')
#             ) fsm on fsm.presented_label_bigint_key = dpl.presented_label_bigint_key

#           where 
#             fsm.company_stock_symbol is not null
#           """).createOrReplaceTempView("dim_company_presented_labels")

In [0]:
# final = spark.sql(f""" 
# select 
# distinct
#     dcpl.report_label_bigint_key as report_label_bigint_key
#     ,dcpl.presented_label_bigint_key as presented_label_bigint_key
#     ,bss.company_bigint_key as company_bigint_key
#     ,bss.report_label as report_label
#     ,bss.date_key as date_key
#     ,dcpl.presented_label as presented_label
#     ,dcpl.company_stock_symbol as company_stock_symbol
# from 
#     balance_sheet_schema bss
# left join 
#     dim_company_presented_labels dcpl on 
# (
#   dcpl.report_label_bigint_key=bss.report_label_bigint_key
#   and 
#   dcpl.company_bigint_key = bss.company_bigint_key
# )""")

In [0]:
# (
#     final
#     .write
#     .format("delta")
#     .mode("overwrite")
#     .option("overwriteSchema", "true")
#     .saveAsTable(f"{catalog}.{schema}.{table_name}")
# )